In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow import keras

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc

In [50]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.metrics import AUC # Area under the curve, default: ROC
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, LearningRateScheduler
import kerastuner as kt

In [4]:
df = pd.read_csv('../databases/diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
def replace_outliers_zeros(data_frame, outlayers_keys: dict, zero_keys: list, mean_median = False):
    """
    Fuction to replace outliers from data_frame
    and replace them with median or mean value
    Don't afect the original data frame, returns a copy 
    with the parameters changed
    """
    data = data_frame.copy()
    if len(zero_keys) != 0:
        for key in zero_keys:
            data.loc[data[key] == 0, key] = np.NaN
            
    replace = {}
    for x in data.keys():
        if mean_median:
            replace[x] = np.mean(data[x].dropna())
        else:
            replace[x] = np.median(data[x].dropna())
    
    for key, val in outlayers_keys.items():
        data.loc[data[key] < val[0], key] = np.NaN
        data.loc[data[key] > val[1], key] = np.NaN
    
    for key, val in replace.items():
        data[key] = data[key].replace(np.NaN, replace[key])
        
    return data

def normalize(data_frame):
    data = data_frame.copy()
    for key in data:
        k_mean = np.mean(data[key])
        k_std = np.std(data[key])
        data[key] = (data[key]-k_mean)/k_std
    return data

def metrics(tn, fp, fn, tp):
    spec = tn / (tn + fp + 1e-15)
    neg_pred_val = tn / (tn + fn + 1e-15)
    sens = tp / (tp + fn + 1e-15)
    pos_pred_val = tp / (tp + fp + 1e-15)
    print('\tEspecificidad: {}\n\tSesitividad: {}\n\tValor predictivo positivo: {}\n\tValor predictivo negativo: {}'.format(spec, 
                                                                                                                    sens, 
                                                                                                                    neg_pred_val, 
                                                                                                                    pos_pred_val))
    
def print_metrics(y, y_pred):
    roc_auc = roc_auc_score(y, y_pred)
    print('PRINCIPAL\n\tArea bajo la curva ROC: {}\nSECUNDARIAS'.format(roc_auc))
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    metrics(tn, fp, fn, tp)
    

Tomando el EDA del Tp1, se reemplazan los valores no medidos por NaN y se quitan los outlayers para luego reemplazarlos por la media o la mediana

In [7]:
outlayers = {
    'BloodPressure': (40, np.Inf),
    'SkinThickness': (0, 80),
    'Insulin': (0, 400),
    'BMI': (0, 50)
}

zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]

In [8]:
x_data = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]
y_data = df['Outcome']

In [9]:
# Split dataset into 15% test, 85% train 
x, x_test, y, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=0)

In [10]:
print('shape train {}, {} '.format(x_train.shape, y_train.shape))
print('shape valid {}, {} '.format(x_valid.shape, y_valid.shape))
print('shape test {}, {} '.format(x_test.shape, y_test.shape))

shape train (554, 8), (554,) 
shape valid (98, 8), (98,) 
shape test (116, 8), (116,) 


In [34]:
x_train_clean = replace_outliers_zeros(x_train, outlayers, zeros, mean_median=True)
x_test_clean = replace_outliers_zeros(x_test, outlayers, zeros, mean_median=True)
x_valid_clean = replace_outliers_zeros(x_valid, outlayers, zeros, mean_median=True)

In [35]:
x_train_clean = normalize(x_train_clean)
x_valid_clean = normalize(x_valid_clean)
x_test_clean = normalize(x_test_clean)

In [36]:
x_train_clean.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,5.540000e+02,5.540000e+02,5.540000e+02,5.540000e+02,5.540000e+02,5.540000e+02,5.540000e+02,5.540000e+02
mean,1.042087e-17,-1.014832e-15,1.087177e-16,5.259331e-15,-1.890185e-15,-1.483771e-15,3.631271e-16,-8.296612e-17
std,1.000904e+00,1.000904e+00,1.000904e+00,1.000904e+00,1.000904e+00,1.000904e+00,1.000904e+00,1.000904e+00
min,-1.132323e+00,-2.551563e+00,-2.762256e+00,-2.604498e+00,-2.319631e+00,-2.181602e+00,-1.226701e+00,-1.039051e+00
25%,-8.395991e-01,-7.297063e-01,-7.318759e-01,-4.614834e-01,-4.475001e-01,-7.538432e-01,-6.903276e-01,-7.910755e-01
50%,-2.541518e-01,-1.500246e-01,-1.908640e-02,1.504268e-02,1.786630e-01,1.376913e-02,-2.925762e-01,-3.777826e-01
75%,6.240192e-01,6.284051e-01,5.868243e-01,3.719111e-01,1.786630e-01,6.547254e-01,4.895369e-01,6.141204e-01
max,3.843979e+00,2.549636e+00,4.103358e+00,3.705489e+00,4.313060e+00,2.700412e+00,5.865088e+00,3.920464e+00


In [14]:
def create_model(in_shape, optimizer_, metrics_, loss_, activation_='sigmoid'):
    model = Sequential()
    model.add(Dense(units=1, activation=activation_, input_shape=in_shape))
    model.compile(optimizer=optimizer_,
                  loss= loss_,
                  metrics=metrics_)
    return model

In [15]:
model = create_model(in_shape= (x_train_clean.shape[1],),
                     optimizer_= SGD(momentum=0.9, learning_rate=0.1),
                     metrics_ = [AUC()],
                     loss_ = BinaryCrossentropy(from_logits=True))

In [16]:
model.fit(x_train_clean, y_train, epochs=20,shuffle=True, validation_data=(x_valid_clean, y_valid))

Epoch 1/20
18/18 [==============================] - 0s 16ms/step - loss: 0.7669 - auc: 0.6120 - val_loss: 0.7769 - val_auc: 0.6787
Epoch 2/20
18/18 [==============================] - 0s 2ms/step - loss: 0.7081 - auc: 0.7708 - val_loss: 0.7388 - val_auc: 0.7287
Epoch 3/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6849 - auc: 0.8142 - val_loss: 0.7216 - val_auc: 0.7488
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6762 - auc: 0.8270 - val_loss: 0.7111 - val_auc: 0.7577
Epoch 5/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6704 - auc: 0.8346 - val_loss: 0.7082 - val_auc: 0.7533
Epoch 6/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6677 - auc: 0.8381 - val_loss: 0.7057 - val_auc: 0.7507
Epoch 7/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6657 - auc: 0.8390 - val_loss: 0.7047 - val_auc: 0.7535
Epoch 8/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6643 - auc: 0.8

In [17]:
print('Sobre train')
y_train_pred = model.predict(x_train_clean)
print_metrics(y_train, np.rint(y_train_pred))
print('Sobre validation')
y_valid_pred = model.predict(x_valid_clean)
print_metrics(y_valid, np.rint(y_valid_pred))

Sobre train
PRINCIPAL
	Area bajo la curva ROC: 0.7145804387183697
SECUNDARIAS
	Especificidad: 0.9316239316239316
	Sesitividad: 0.4975369458128079
	Valor predictivo positivo: 0.7622377622377622
	Valor predictivo negativo: 0.808
Sobre validation
PRINCIPAL
	Area bajo la curva ROC: 0.6799687010954617
SECUNDARIAS
	Especificidad: 0.9154929577464789
	Sesitividad: 0.4444444444444444
	Valor predictivo positivo: 0.8125
	Valor predictivo negativo: 0.6666666666666666


In [18]:
model = create_model(in_shape= (x_train_clean.shape[1],),
                     optimizer_= Adam(learning_rate=0.1),
                     metrics_ = [AUC()],
                     loss_ = BinaryCrossentropy(from_logits=True))

In [19]:
model.fit(x_train_clean, y_train, epochs=20,shuffle=True, validation_data=(x_valid_clean, y_valid))

Epoch 1/20
18/18 [==============================] - 0s 11ms/step - loss: 0.7289 - auc_1: 0.7004 - val_loss: 0.7338 - val_auc_1: 0.7144
Epoch 2/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6697 - auc_1: 0.8273 - val_loss: 0.7040 - val_auc_1: 0.7467
Epoch 3/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6595 - auc_1: 0.8320 - val_loss: 0.7006 - val_auc_1: 0.7572
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6579 - auc_1: 0.8316 - val_loss: 0.7012 - val_auc_1: 0.7514
Epoch 5/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6563 - auc_1: 0.8323 - val_loss: 0.7014 - val_auc_1: 0.7587
Epoch 6/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6562 - auc_1: 0.8278 - val_loss: 0.6986 - val_auc_1: 0.7548
Epoch 7/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6562 - auc_1: 0.8190 - val_loss: 0.6974 - val_auc_1: 0.7598
Epoch 8/20
18/18 [==============================] - 0s 2ms/st

In [20]:
print('Sobre train')
y_train_pred = model.predict(x_train_clean)
print_metrics(y_train, np.rint(y_train_pred))
print('Sobre validation')
y_valid_pred = model.predict(x_valid_clean)
print_metrics(y_valid, np.rint(y_valid_pred))

Sobre train
PRINCIPAL
	Area bajo la curva ROC: 0.7307832652660239
SECUNDARIAS
	Especificidad: 0.9344729344729344
	Sesitividad: 0.5270935960591133
	Valor predictivo positivo: 0.7735849056603774
	Valor predictivo negativo: 0.823076923076923
Sobre validation
PRINCIPAL
	Area bajo la curva ROC: 0.6914449660928533
SECUNDARIAS
	Especificidad: 0.9014084507042254
	Sesitividad: 0.48148148148148145
	Valor predictivo positivo: 0.8205128205128205
	Valor predictivo negativo: 0.65


In [21]:
def model_builder(hp):
    model = Sequential()
    model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_clean.shape[1],)))
    
    # Optimizadores a porbar
    optimizer_type = hp.Choice('optimizer_type', values=['SGD', 'Adam']) 
    # Hyperparametros para porbar en el optimizador
    _learning_rate_ = hp.Choice('learning_rate', values=[1e-3, 1e-2, 0.1, 0.32, 0.56, 0.81, 1.0, 1.2, 1.5, 1.8, 2.0, 2.2, 2.7])
    _momentum_ = hp.Choice('momentum', values=[0.5, 0.75, 0.85, 0.9, 0.99, 0.999])
    
    if optimizer_type == 'SGD':
        _optimizer_ = SGD(learning_rate=_learning_rate_, momentum=_momentum_)
    else:
        _optimizer_ = Adam(learning_rate=_learning_rate_)
    
    _metrics_ = [AUC()]
    _loss_ = BinaryCrossentropy(from_logits=True)
    
    model.compile(optimizer=_optimizer_,
                  loss=_loss_,
                  metrics=_metrics_)
    
    return model


In [22]:
hyptester = kt.Hyperband(model_builder, kt.Objective('val_loss', direction='min'), max_epochs=15, factor=3)

In [23]:
hyptester.search(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid))
hyperparams = hyptester.get_best_hyperparameters(num_trials=2)[0]

Trial 30 Complete [00h 00m 01s]
val_loss: 0.6863452196121216

Best val_loss So Far: 0.6806035041809082
Total elapsed time: 00h 00m 52s
INFO:tensorflow:Oracle triggered exit


In [24]:
print('best momentum',hyperparams.get('momentum'))
print('best learning_rate',hyperparams.get('learning_rate'))
print('best optimizer', hyperparams.get('optimizer_type'))

best momentum 0.9
best learning_rate 2.0
best optimizer Adam


In [25]:
best_hyp_model = hyptester.hypermodel.build(hyperparams)
best_hyp_model.fit(x_train_clean, y_train, epochs=20,shuffle=True, validation_data=(x_valid_clean, y_valid))

Epoch 1/20
18/18 [==============================] - 0s 12ms/step - loss: 0.7063 - auc: 0.7052 - val_loss: 0.7265 - val_auc: 0.6693
Epoch 2/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6705 - auc: 0.7225 - val_loss: 0.7159 - val_auc: 0.6641
Epoch 3/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6624 - auc: 0.7179 - val_loss: 0.6900 - val_auc: 0.6575
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6562 - auc: 0.7108 - val_loss: 0.6885 - val_auc: 0.6552
Epoch 5/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6610 - auc: 0.6901 - val_loss: 0.6840 - val_auc: 0.7136
Epoch 6/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6588 - auc: 0.7018 - val_loss: 0.6835 - val_auc: 0.6787
Epoch 7/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6614 - auc: 0.6793 - val_loss: 0.6842 - val_auc: 0.6390
Epoch 8/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6548 - auc: 0.7

In [26]:
print('Sobre train')
y_train_pred = best_hyp_model.predict(x_train_clean)
print_metrics(y_train, np.rint(y_train_pred))
print('Sobre validation')
y_valid_pred = best_hyp_model.predict(x_valid_clean)
print_metrics(y_valid, np.rint(y_valid_pred))

Sobre train
PRINCIPAL
	Area bajo la curva ROC: 0.7428599497565015
SECUNDARIAS
	Especificidad: 0.8945868945868946
	Sesitividad: 0.5911330049261084
	Valor predictivo positivo: 0.7909319899244333
	Valor predictivo negativo: 0.7643312101910829
Sobre validation
PRINCIPAL
	Area bajo la curva ROC: 0.6632759520083463
SECUNDARIAS
	Especificidad: 0.8450704225352113
	Sesitividad: 0.48148148148148145
	Valor predictivo positivo: 0.8108108108108109
	Valor predictivo negativo: 0.5416666666666666


# Early Stopping

Detiene el entrenamiento de forma anticipada si se llega a cierto valor de un parametro determinado, en principio permite agilizar las pruebas al buscar hyperparámetros, descartanto rapidamenete las combinacion que converjan rapidamente a un valor esperado

Los hyperparámetrso principales de este callback son:
- monitor: indica la variable a monitorear (valga la redundancia)
- min_delta: la minima diferencia que se considera como mejora
- patience: cantidad de epochs sin mejoras antes de parar

In [47]:
stop_early = EarlyStopping(monitor='val_loss', patience=15)

In [48]:
hyptester_es = kt.Hyperband(model_builder, kt.Objective('val_loss', direction='min'), max_epochs=50, factor=3)
hyptester_es.search(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid), callbacks=[stop_early])
# Get the optimal hyperparameters
hyperparams_es = hyptester_es.get_best_hyperparameters(num_trials=2)[0]
print('best momentum',hyperparams_es.get('momentum'))
print('best learning_rate',hyperparams_es.get('learning_rate'))
print('best optimizer', hyperparams_es.get('optimizer_type'))

Trial 90 Complete [00h 00m 02s]
val_loss: 0.6901907920837402

Best val_loss So Far: 0.6796790361404419
Total elapsed time: 00h 03m 46s
INFO:tensorflow:Oracle triggered exit
best momentum 0.99
best learning_rate 1.0
best optimizer SGD


In [53]:
best_hyp_model_es = hyptester_es.hypermodel.build(hyperparams_es)
best_hyp_model_es.fit(x_train_clean, y_train, epochs=50,shuffle=True, validation_data=(x_valid_clean, y_valid), callbacks=[stop_early])
print('Sobre train')
y_train_pred = best_hyp_model_es.predict(x_train_clean)
print_metrics(y_train, np.rint(y_train_pred))
print('Sobre validation')
y_valid_pred = best_hyp_model_es.predict(x_valid_clean)
print_metrics(y_valid, np.rint(y_valid_pred))

Epoch 1/50
18/18 [==============================] - 0s 17ms/step - loss: 0.7022 - auc: 0.7445 - val_loss: 0.7027 - val_auc: 0.7055
Epoch 2/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6587 - auc: 0.7729 - val_loss: 0.6916 - val_auc: 0.6440
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6607 - auc: 0.7365 - val_loss: 0.6896 - val_auc: 0.6528
Epoch 4/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6570 - auc: 0.7199 - val_loss: 0.6859 - val_auc: 0.6492
Epoch 5/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6532 - auc: 0.7232 - val_loss: 0.6854 - val_auc: 0.6505
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6533 - auc: 0.7307 - val_loss: 0.6910 - val_auc: 0.6560
Epoch 7/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6531 - auc: 0.7342 - val_loss: 0.6966 - val_auc: 0.6724
Epoch 8/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6523 - auc: 0.7

Se puede ver que los resultados sobre el set de validación son similares, lo que fue beneficioso al utilizar el earlystopping fue la posibilidad de probar una mayor cantidad de combinaciónes de hyperparámetros en un tiempo similar, obteniendo asi una combinación de hyperparámetros más confiable

# Learning Rate Scheduler

Permite ajustar el learning rate en funcion del epoch que se esta trabajando, por lo tanto dependiendo la funcion de costos minimizarla de forma más rapida y eficiente???

In [61]:
def schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * np.exp(-0.8)

In [62]:
scheduler = LearningRateScheduler(schedule)

In [64]:
best_hyp_model_es_sched = hyptester_es.hypermodel.build(hyperparams_es)
best_hyp_model_es_sched.fit(x_train_clean, y_train, epochs=50,shuffle=True, validation_data=(x_valid_clean, y_valid), 
                            callbacks=[stop_early, scheduler])
print('\nSobre train')
y_train_pred = best_hyp_model_es_sched.predict(x_train_clean)
print_metrics(y_train, np.rint(y_train_pred))
print('Sobre validation')
y_valid_pred = best_hyp_model_es_sched.predict(x_valid_clean)
print_metrics(y_valid, np.rint(y_valid_pred))

Epoch 1/50
18/18 [==============================] - 0s 17ms/step - loss: 0.6874 - auc: 0.7871 - val_loss: 0.6933 - val_auc: 0.7293
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6569 - auc: 0.7847 - val_loss: 0.6823 - val_auc: 0.6815
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6562 - auc: 0.7611 - val_loss: 0.6831 - val_auc: 0.6896
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6532 - auc: 0.7454 - val_loss: 0.6836 - val_auc: 0.6883
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6503 - auc: 0.7339 - val_loss: 0.6842 - val_auc: 0.6896
Epoch 6/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6515 - auc: 0.7309 - val_loss: 0.6867 - val_auc: 0.6886
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6513 - auc: 0.7287 - val_loss: 0.6891 - val_auc: 0.6886
Epoch 8/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6509 - auc: 0.7

# Model CheckPoint & Tensorboard

# Tecnicas de Regularización

*   Drop out

*   Regularización L1
*   Regularización L2
*   Batch Normalization
